In [1]:
import pandas as pd
import os, re
import numpy as np
import math

from mhystic.preprocessing import log_meas, select_by_allele, meas2binary, unify_alleles

In [2]:
path = "./models_local/cnn.128.64_bal/"
hists = os.listdir(path)[1:-3]
lists = []
for i in hists:
    lists.append(i.split("."))
    
alleles = np.array(lists)[:, 2]

f1s = {}
for i, j in zip(hists, alleles):
    cur_f1 = np.genfromtxt(path+i)[-1]
    f1s[j.split(",")[0]] = cur_f1

In [3]:
bdata = pd.read_csv("./data/bdata.2009.tsv", sep="\t")
blind = pd.read_csv("./data/blind.tsv", sep="\t")

bdata.mhc = bdata.mhc.apply(unify_alleles)
blind.mhc = blind.mhc.apply(unify_alleles)

bdata.meas = bdata.meas.apply(log_meas)
blind.meas = blind.meas.apply(log_meas)

In [4]:
f1_df = pd.DataFrame(columns=["mhc", "f1", "bind_train", "nonbind_train", "bind_test", "nonbind_test"])

for cur_allele in list(f1s.keys()):
    
    cur_data_train = select_by_allele(bdata, cur_allele)
    cur_data_test = select_by_allele(blind, cur_allele)
    
    binary_meas_train = np.array(meas2binary(cur_data_train.meas))
    binary_meas_test = np.array(meas2binary(cur_data_test.meas))
    
    zeros_train, ones_train = len(np.where(binary_meas_train == False)[0]), len(np.where(binary_meas_train == True)[0])
    zeros_test, ones_test = len(np.where(binary_meas_test == False)[0]), len(np.where(binary_meas_test == True)[0])

    cur_row = [cur_allele, f1s[cur_allele], ones_train, zeros_train, ones_test, zeros_test]
    f1_df.loc[len(f1_df)] = cur_row

In [5]:
f1_df

mhc        f1  bind_train  nonbind_train  bind_test  nonbind_test
0   HLAA0101  0.666667       512.0         3213.0       99.0         597.0
1   HLAA0201  0.890739      3150.0         6415.0     1148.0         978.0
2   HLAA0202  0.760870      1758.0         2161.0       44.0          82.0
3   HLAA0203  0.932540      1953.0         3589.0      426.0         225.0
4   HLAA0206  0.817248      1797.0         3030.0      419.0         263.0
5   HLAA0301  0.870482      1533.0         4608.0      486.0         325.0
6   HLAA1101  0.861985      1758.0         3641.0      341.0         382.0
7   HLAA2301  0.750000       398.0         1623.0      172.0         219.0
8   HLAA2402  0.701031       420.0         2113.0      232.0         341.0
9   HLAA2501  0.500000        66.0          453.0        5.0         411.0
10  HLAA2601  0.561404       319.0         2575.0      130.0        1203.0
11  HLAA2902  0.666667       643.0         1754.0       27.0          91.0
12  HLAA3001  0.786517       574.0         1466.0      183.0         477.0
13  HLAA3002  0.663968       395.0         1035.0      183.0         237.0
14  HLAA3101  0.833652      1146.0         3650.0      386.0         338.0
15  HLAA3201  0.770270       279.0          361.0      131.0         318.0
16  HLAA3301  0.873065       416.0         2624.0      288.0         185.0
17  HLAA6801  0.883117      1222.0         1962.0      349.0         178.0
18  HLAA6802  0.898630       967.0         3801.0      355.0         314.0
19  HLAA6901  0.554217       222.0         1857.0       26.0         444.0
20  HLAA8001  0.357143       113.0          669.0        7.0         372.0
21  HLAB0702  0.898123       686.0         2726.0      486.0         327.0
22  HLAB0801  0.849246       482.0         1785.0      249.0         691.0
23  HLAB1501  0.821918       844.0         2369.0      215.0         418.0
24  HLAB1517  0.658228       271.0          575.0       93.0         489.0
25  HLAB1801  0.368421       248.0         1804.0       28.0         475.0
26  HLAB2705  0.378378       568.0         2460.0       29.0         285.0
27  HLAB3501  0.766667       585.0         1812.0      223.0         319.0
28  HLAB3901  0.734694       170.0          716.0       25.0         616.0
29  HLAB4001  0.848214       337.0         2381.0      198.0         209.0
30  HLAB4402  0.684444       205.0         1500.0      230.0         181.0
31  HLAB4403  0.682243       209.0          704.0      199.0         179.0
32  HLAB4601  0.000000        93.0         1331.0        0.0         378.0
33  HLAB5101  0.500000       217.0         1517.0       64.0         790.0
34  HLAB5301  0.748148       290.0          728.0      206.0         279.0
35  HLAB5401  0.800000       207.0          812.0        5.0          74.0
36  HLAB5701  0.780669       234.0         1623.0      198.0         617.0
37  HLAB5801  0.927445       417.0         2147.0      219.0         226.0

In [6]:
f1_df.to_csv("./data/local_models_f1s.csv", index=False)